In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline
import matplotlib
#matplotlib.use('agg')
matplotlib.style.use('ggplot')
import pickle as pkl
from matplotlib import pyplot as plt
from collections import Counter
from functools import reduce
import random
random.seed(1991)
import glob

In [4]:
heads=pd.read_pickle("/data/dharp/compounding/datasets/heads_CompoundCentric_DecadeAgnostic_300.pkl")
#heads.reset_index(inplace=True)
#heads=heads.drop(['decade'],axis=1).groupby(['head']).mean()
heads.info()
heads.head()

<class 'pandas.core.frame.DataFrame'>
Index: 53466 entries, a_n to zygote_n
Columns: 300 entries, 0 to 299
dtypes: float64(300)
memory usage: 122.8+ MB


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
head,,,,,,,,,,,,,,,,,,,,,
a_n,0.440,0.006,0.002,-0.003,-0.001,-0.025,0.195,-0.031,-0.010,-0.104,...,-0.028,0.027,-0.044,0.007,0.008,0.015,0.019,-0.012,-0.016,0.013
aa_n,0.012,0.002,-0.001,0.001,0.000,-0.003,0.050,-0.017,0.000,0.051,...,-0.038,-0.033,-0.009,-0.001,0.014,0.026,0.050,-0.027,0.003,0.019
aaa_n,0.313,-0.016,0.012,0.003,-0.001,-0.017,0.154,-0.031,0.108,-0.081,...,-0.046,-0.058,-0.012,0.011,0.036,0.029,0.030,-0.039,0.025,0.017
aaaaa_n,0.001,0.000,0.000,0.000,-0.000,-0.000,0.004,-0.001,0.001,-0.005,...,-0.045,-0.023,-0.012,0.016,0.013,0.028,0.048,0.008,0.025,0.013
aaai_n,0.001,0.001,0.000,0.002,0.005,0.002,0.004,-0.000,0.034,-0.004,...,0.023,0.041,-0.096,0.044,-0.010,-0.068,0.141,0.148,-0.015,0.071


In [5]:
modifiers=pd.read_pickle("/data/dharp/compounding/datasets/modifiers_CompoundCentric_DecadeAgnostic_300.pkl")
#modifiers.reset_index(inplace=True)
#modifiers=modifiers.drop(['decade'],axis=1).groupby(['modifier']).mean()
modifiers.info()
modifiers.head()

<class 'pandas.core.frame.DataFrame'>
Index: 54400 entries, a_n to zylobalsamum_n
Columns: 300 entries, 0 to 299
dtypes: float64(300)
memory usage: 124.9+ MB


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
modifier,,,,,,,,,,,,,,,,,,,,,
a_n,0.340,0.033,-0.002,0.122,-0.022,-0.016,0.114,-0.026,0.053,-0.093,...,-0.026,-0.043,0.014,0.023,-0.039,0.036,0.037,0.013,0.017,-0.006
aa_n,0.192,0.012,0.002,-0.002,0.037,-0.006,0.051,-0.015,0.002,-0.046,...,-0.057,-0.030,-0.027,0.010,0.044,0.018,0.062,0.030,0.025,-0.001
aaa_n,0.065,0.047,0.000,0.091,-0.016,-0.004,0.004,-0.002,-0.006,-0.005,...,-0.040,-0.024,-0.025,0.013,0.022,0.035,0.047,0.000,0.022,-0.000
aaaa_n,0.001,0.000,0.000,0.000,-0.000,-0.000,0.004,-0.001,0.001,-0.005,...,-0.045,-0.023,-0.012,0.016,0.013,0.028,0.048,0.008,0.025,0.013
aaaaa_n,0.001,0.000,0.000,0.000,-0.000,-0.000,0.004,-0.001,0.001,-0.005,...,-0.045,-0.023,-0.012,0.016,0.013,0.028,0.048,0.008,0.025,0.013


In [6]:
novel_compounds_list = pkl.load( open( "/data/dharp/compounding/datasets/novel_compounds_list.pkl", "rb" ) )
m, h = zip(*novel_compounds_list)
heads_list=list(set(h))
modifiers_list=list(set(m))

In [7]:
novel_compounds=pd.DataFrame(novel_compounds_list)
novel_compounds.columns=['modifier','head']
novel_compounds

,modifier,head
0,a_n,peaceful_n
1,a_n,robinson_n
2,a_n,supply_n
3,india_n,peaceful_n
4,india_n,standard_n
5,india_n,award_n
6,india_n,joint_n
7,india_n,companion_n
8,india_n,foundation_n
9,india_n,gentleman_n


In [8]:
positive_df=pd.merge(novel_compounds,heads.reset_index(),on=["head"])
positive_df=pd.merge(positive_df,modifiers.reset_index(),on=["modifier"])
positive_df['Plausibility']=True
positive_df

,modifier,head,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,...,291_y,292_y,293_y,294_y,295_y,296_y,297_y,298_y,299_y,Plausibility
0,a_n,peaceful_n,0.095,0.395,0.049,-0.055,0.067,0.017,-0.067,-0.006,...,-0.043,0.014,0.023,-0.039,0.036,0.037,0.013,0.017,-0.006,True
1,a_n,robinson_n,0.517,-0.028,-0.009,0.022,-0.003,-0.024,0.113,-0.010,...,-0.043,0.014,0.023,-0.039,0.036,0.037,0.013,0.017,-0.006,True
2,a_n,supply_n,0.849,-0.050,-0.024,-0.010,0.010,-0.052,0.358,-0.056,...,-0.043,0.014,0.023,-0.039,0.036,0.037,0.013,0.017,-0.006,True
3,india_n,peaceful_n,0.095,0.395,0.049,-0.055,0.067,0.017,-0.067,-0.006,...,0.010,0.008,-0.001,0.004,0.015,-0.007,-0.002,-0.010,0.000,True
4,india_n,standard_n,0.488,0.014,-0.005,0.044,0.058,-0.025,0.290,-0.080,...,0.010,0.008,-0.001,0.004,0.015,-0.007,-0.002,-0.010,0.000,True
5,india_n,award_n,0.468,-0.019,-0.005,0.019,0.019,-0.023,0.154,-0.020,...,0.010,0.008,-0.001,0.004,0.015,-0.007,-0.002,-0.010,0.000,True
6,india_n,joint_n,0.716,-0.036,-0.016,-0.007,-0.003,-0.038,0.313,-0.058,...,0.010,0.008,-0.001,0.004,0.015,-0.007,-0.002,-0.010,0.000,True
7,india_n,companion_n,0.648,-0.019,-0.006,0.087,-0.013,-0.023,0.229,0.007,...,0.010,0.008,-0.001,0.004,0.015,-0.007,-0.002,-0.010,0.000,True
8,india_n,foundation_n,0.234,0.024,-0.011,0.125,0.018,-0.005,0.016,0.003,...,0.010,0.008,-0.001,0.004,0.015,-0.007,-0.002,-0.010,0.000,True
9,india_n,gentleman_n,0.858,-0.063,-0.028,0.011,-0.001,-0.041,0.202,-0.035,...,0.010,0.008,-0.001,0.004,0.015,-0.007,-0.002,-0.010,0.000,True


In [9]:
def neg_df_creator(file):
    pkl_file=pkl.load( open(file,'rb'))
    df=pd.DataFrame(pkl_file)
    df.columns=['modifier','head']
    negative_df=pd.merge(df,heads.reset_index(),on=["head"])
    negative_df=pd.merge(negative_df,modifiers.reset_index(),on=["modifier"])
    negative_df['Plausibility']=False
    return negative_df

In [10]:
def df_joiner(files):
    df_list=[]
    for file in files:
        neg_df=neg_df_creator(file)
        whole_df=pd.concat([neg_df,positive_df])
        df_list.append(whole_df)
    
    return df_list

In [11]:
corrupt_modifier_files=[]
for file in glob.glob("/data/dharp/compounding/datasets/corrupt_modifier*"):
    corrupt_modifier_files.append(file)
corrupt_modifiers=df_joiner(corrupt_modifier_files)

In [12]:
corrupt_head_files=[]
for file in glob.glob("/data/dharp/compounding/datasets/corrupt_head*"):
    corrupt_head_files.append(file)
corrupt_heads=df_joiner(corrupt_head_files)

In [13]:
acc_ch=[]
for i,corrupt_head in enumerate(corrupt_heads):
    data=corrupt_head.drop(['modifier','head'],axis=1)
    X, Y = data.iloc[:,:-1],data.iloc[:,-1]
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1991)
    # fit model no training data
    model = XGBClassifier(njobs=-1)
    model.fit(X_train, y_train)
    # make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    print("DF",i+1)
    accuracy = accuracy_score(y_test, predictions)
    acc_ch.append(accuracy)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))


DF 1
Accuracy: 68.11%
DF 2
Accuracy: 68.48%
DF 3
Accuracy: 67.99%
DF 4
Accuracy: 68.58%
DF 5
Accuracy: 69.13%
DF 6
Accuracy: 68.43%
DF 7
Accuracy: 67.48%
DF 8
Accuracy: 68.05%
DF 9
Accuracy: 68.45%
DF 10
Accuracy: 67.88%


In [14]:
round(np.mean(acc_ch)*100,2)

68.26

In [15]:
round(np.std(acc_ch)*100,2)

0.43

In [16]:
acc_cm=[]
for i,corrupt_head in enumerate(corrupt_modifiers):
    data=corrupt_head.drop(['modifier','head'],axis=1)
    X, Y = data.iloc[:,:-1],data.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1991)
# fit model no training data
    model = XGBClassifier(njobs=-1)
    model.fit(X_train, y_train)
# make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
# evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    acc_cm.append(accuracy)
    print("DF",i+1)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))

DF 1
Accuracy: 65.73%
DF 2
Accuracy: 65.25%
DF 3
Accuracy: 64.45%
DF 4
Accuracy: 65.25%
DF 5
Accuracy: 65.25%
DF 6
Accuracy: 64.96%
DF 7
Accuracy: 64.75%
DF 8
Accuracy: 64.93%
DF 9
Accuracy: 65.06%
DF 10
Accuracy: 64.75%


In [17]:
round(np.mean(acc_cm)*100,2)

65.04

In [18]:
round(np.std(acc_cm)*100,2)

0.34